In [23]:
# get_data.py
import os
import pandas as pd
from dotenv import load_dotenv
from entsoe import EntsoePandasClient
import plotly.graph_objects as go

load_dotenv()
entsoe_api_key = os.getenv("ENTSOE_API_KEY")

start = pd.Timestamp("20220601", tz="Europe/Brussels")
end = pd.Timestamp("20231201", tz="Europe/Brussels")
country_code = "UK"
output_file_name = "entose_train_data_load.json"


client = EntsoePandasClient(api_key=entsoe_api_key)
df = client.query_load(country_code, start=start, end=end)
# df = client.query_day_ahead_prices(country_code, start=start, end=end)
display(df)
# fig = go.Figure()
# fig.add_trace(
#     go.Scatter(
#         x=df.index,
#         y=df["Actual Load"],
#         mode="lines",
#         name="Energy Prices",
#         line=dict(color="blue"),
#     )
# )
# fig.update_layout(
#     title="Energy Prices in London (Past 3 Months)",
#     xaxis_title="Datetime",
#     yaxis_title="Price (Euro/MWh)",
# )
# fig.show()


with open(output_file_name, "w") as file:
    file.write(df.to_json())

KeyboardInterrupt: 

In [26]:
import tensorflow as tf

print(tf.__version__)

2.15.0


In [25]:
# save_model.py
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices"


import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


# Load data
data_filename = "entose_train_data_load.json"
df = pd.read_json(data_filename)
# display(df)

# Preprocess data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_load = scaler.fit_transform(df.values.reshape(-1, 1))

# Reshape for LSTM, y is the next half an hour load
X = []
y = []
for i in range(1, len(scaled_load)):
    X.append(scaled_load[i - 1 : i, 0])
    y.append(scaled_load[i, 0])
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

# Train
model = Sequential()
model.add(LSTM(50, activation="tanh", input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20


2023-12-17 18:42:29.975690: W external/local_xla/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:542] libdevice is required by this HLO module but was not found at ./libdevice.10.bc
2023-12-17 18:42:29.977668: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:574 : INTERNAL: libdevice not found at ./libdevice.10.bc
2023-12-17 18:42:30.003805: W external/local_xla/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:542] libdevice is required by this HLO module but was not found at ./libdevice.10.bc
2023-12-17 18:42:30.005317: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at xla_ops.cc:574 : INTERNAL: libdevice not found at ./libdevice.10.bc


InternalError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_4 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/mesonyktio/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/mesonyktio/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1077, in launch_instance

  File "/home/mesonyktio/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/mesonyktio/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/mesonyktio/.local/lib/python3.10/site-packages/ipykerne

In [28]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pickle


# Load
data_filename = "entose_train_data_load.json"
df = pd.read_json(data_filename)

# Preprocess
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_load = scaler.fit_transform(df.values.reshape(-1, 1))

# Reshape
X = []
y = []
for i in range(1, len(scaled_load)):
    X.append(scaled_load[i - 1 : i, 0])
    y.append(scaled_load[i, 0])

X, y = np.array(X), np.array(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

model = xgb.XGBRegressor(
    objective="reg:squarederror",
    colsample_bytree=0.3,
    learning_rate=0.1,
    max_depth=5,
    alpha=10,
    n_estimators=100,
)

model.fit(X_train, y_train)

# Evaluate
predictions = model.predict(X_test)

# Save
model_filename = "xgboost_model.pkl"
pickle.dump(model, open(model_filename, "wb"))

In [1]:
import pickle
from mlserver.types import InferenceRequest

X_test_file = "X_test.pkl"
y_test_file = "y_test.pkl"
X_test = pickle.load(open(X_test_file, "rb"))
y_test = pickle.load(open(y_test_file, "rb"))
X_0 = X_test[0]
print(f"{X_0 = }")
# print(f"{InferenceRequest(X_0) = }")
# InferenceRequest(X_0.tolist())
X_0.reshape(1, -1)

X_0 = array([0.4058296])


array([[0.4058296]])

In [4]:
model = pickle.load(open(model_filename, "rb"))
model.predict(X_0.reshape(1, -1))

array([0.4037375], dtype=float32)

In [13]:
from mlserver.types import InferenceRequest, RequestInput
import requests
import json


test_sample = X_test[0].tolist()
input_data = {
    "name": "X_test",
    "shape": [len(test_sample)],
    "datatype": "FP32",
    "data": test_sample,
}
payload = {"inputs": [input_data]}
response = requests.post(
    "http://localhost:8080/v2/models/xgboost-model/infer",
    data=json.dumps(payload),
)
print(response.json())

{'error': 'Model xgboost-model not found'}
